## Changing components in a Mimi model

This is meant as a simple hands-on example that can be readily executed and adapted. Some more documentation examples can be found [here](https://www.mimiframework.org/Mimi.jl/stable/tutorials/tutorial_3/#Component-and-Structural-Modifications:-The-API-1).

> NOTE:
> If you prefer this tutorial as a `.jl` file, just copy out the
> code bits and execute them there, with `#%%` to separate cells.

In [ ]:
using Pkg;
Pkg.activate(".") # activate the environment in the "modelling" folder
Pkg.instantiate() # install the packages in the environment

### 1. Modifying a DICE component

Let's change the DICE damage component. Currently, it operates like this:
$\begin{equation}
    DAMFRAC = a_2 T_\text{atm}^2
\end{equation}$
($a_1=0, a_3=2$), or in code:
```julia
v.DAMFRAC[t] = p.a1 * p.TATM[t] + p.a2 * p.TATM[t] ^ p.a3
```
That means, the fraction of climate damages compared to total world GDP in a given timestep is calculated based on the global mean surface temperature of the same timestep. Timesteps in DICE2016 are 5 years long.

Let's assume that damages don't only depend on the given time period, but also to some extent on the previous one. For simplicitly, let's assume that 50% of the climate damages (in relative terms) that occured in the previous time period still affect the economy in the subsequent time period.

In code, this can be done by making the variable `v.DAMFRAC[t]` depend not only on `p.TATM[t]`, but also on `p.TATM[t-1]`. The adapted equation (in code) looks like this:
```julia
v.DAMFRAC[t] = p.a1 * p.TATM[t] + p.a2 * p.TATM[t] ^ p.a3 + 0.5 * p.a2 * p.TATM[t-1] ^ p.a3 
```
Now there is one complication: what happens in the first time step? To calculate `v.DAMFRAC[1]`, the equation new requires `p.TATM[0]`, which does not exist (because 1 is the first time step). Running the model like this will throw an error.

We can resolve this by explicitly treating the first time step differently:
```julia
if is_first(t)
    v.DAMFRAC[t] = p.a1 * p.TATM[t] + p.a2 * p.TATM[t] ^ p.a3 + 0.5 * p.a2 * p.tatm0 ^ p.a3
else
    v.DAMFRAC[t] = p.a1 * p.TATM[t] + p.a2 * p.TATM[t] ^ p.a3 + 0.5 * p.a2 * p.TATM[t-1] ^ p.a3
end
```
Now, in the first timestep damages depend on a time-independent temperature value for the 0th timestep `p.tatm0` that we will have to set later. In code adding a new parameter to a component will require a line of this form:
```julia
tatm0 = Parameter() # Initial atmospheric temp change (C from 1900)
```

By the way, this structural modification is not meant to be realistic or necessarily super sensible, but mainly to showcase the procedure of replacing a Mimi component.

### 2. Loading and using a modified component

In [ ]:
using Mimi
using MimiDICE2016R2

In [ ]:
m_DICE = MimiDICE2016R2.get_model()
run(m_DICE)

The modified damage component can be loaded with the `include` statemtent and then replace the original `:damages` component. All parameter connections will stay in place.

In [ ]:
include("./modified_damages_component.jl")
replace!(m_DICE, :damages => modified_damages);

The newly introduced parameter `tatm0`, though, still has to be set. The `:climatedynamics` component already has such a `tatm0` parameter, which we now want to connect. To connect two parameters, we can create a shared model parameter `:initial_temp`, which we set to the value of `tatm0` in `:climatedynamics`.

In [ ]:
tatm0 = m_DICE[:climatedynamics, :tatm0]
add_shared_param!(m_DICE, :initial_temp, tatm0)

Having created this shared model parameter, we can connect it to the two `tatm0` parameters in the respective components and run the model with the updated damage equation.

In [ ]:
connect_param!(m_DICE, :damages, :tatm0, :initial_temp)
connect_param!(m_DICE, :climatedynamics, :tatm0, :initial_temp)

In [ ]:
run(m_DICE)